# NIQE and Ma et al. metrics study on satellite imagery

In this notebook we will investigate how the tile size of satellite imagery affects [NIQE](https://ieeexplore.ieee.org/document/6353522) and [Ma et al.](https://www.sciencedirect.com/science/article/pii/S107731421630203X) (short form *Ma*) scores.

We will generate random tile samples of varying size from two satellite sensors, WorldView-2 and GeoEye-1, and use an integration with MATLAB through the [MATLAB Engine API for Python](https://se.mathworks.com/help/matlab/matlab-engine-for-python.html) to evaluate *NIQE* and *Ma et al.* scores

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

from modules.matlab_metrics import *
from modules.image_utils import *
from modules.helpers import *
from modules.tile_generator import *
from modules.losses_metrics import perceptual_index

import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots
#init_notebook_mode(connected=True)

In [29]:
TILES_DIR = 'data/niqe-ma-study/512'

BATCH_SIZE = 1
SHAVE_WIDTH = 4

In [30]:
tile_paths = list(pathlib.Path(TILES_DIR).glob('**/pan/*.tif'))
n = len(tile_paths)
print(n)

1000


In [12]:
matlab_engine = MatLabEngine('modules/matlab', ma=False, niqe=True, 
                             output_dtype='uint16', shave_width=SHAVE_WIDTH)


Starting matlab.engine ...
matlab.engine started


In [31]:
for i, tile_path in enumerate(tile_paths):
    pan_tile = geotiff_to_ndarray(tile_path)
    print(pan_tile.shape)
    pan_tile = np.expand_dims(pan_tile, 0)
    print(pan_tile.shape)
    niqe = matlab_engine.matlab_niqe_metric(pan_tile)
    print(niqe)
    if i >= 10:
        break

(512, 512, 1)
(1, 512, 512, 1)
4.996291822900584
(512, 512, 1)
(1, 512, 512, 1)
11.118760515117486
(512, 512, 1)
(1, 512, 512, 1)
3.971781176728327
(512, 512, 1)
(1, 512, 512, 1)
5.282714987213867
(512, 512, 1)
(1, 512, 512, 1)
4.484912651615964
(512, 512, 1)
(1, 512, 512, 1)
4.8346654027429885
(512, 512, 1)
(1, 512, 512, 1)
4.454226808797914
(512, 512, 1)
(1, 512, 512, 1)
10.783362243150046
(512, 512, 1)
(1, 512, 512, 1)
4.530453587510806
(512, 512, 1)
(1, 512, 512, 1)
10.94642994914038
(512, 512, 1)
(1, 512, 512, 1)
10.923325765923432


In [32]:
for i, tile_path in enumerate(tile_paths):
    pan_tile = geotiff_to_ndarray(tile_path)
    print(pan_tile.shape)
    pan_tile = np.expand_dims(pan_tile, 0)
    print(pan_tile.shape)
    niqe = matlab_engine.matlab_niqe_metric(pan_tile)
    print(niqe)
    if i >= 10:
        break

(512, 512, 1)
(1, 512, 512, 1)
4.996291822900584
(512, 512, 1)
(1, 512, 512, 1)
11.118760515117486
(512, 512, 1)
(1, 512, 512, 1)
3.971781176728327
(512, 512, 1)
(1, 512, 512, 1)
5.282714987213867
(512, 512, 1)
(1, 512, 512, 1)
4.484912651615964
(512, 512, 1)
(1, 512, 512, 1)
4.8346654027429885
(512, 512, 1)
(1, 512, 512, 1)
4.454226808797914
(512, 512, 1)
(1, 512, 512, 1)
10.783362243150046
(512, 512, 1)
(1, 512, 512, 1)
4.530453587510806
(512, 512, 1)
(1, 512, 512, 1)
10.94642994914038
(512, 512, 1)
(1, 512, 512, 1)
10.923325765923432


# Generate tiles with different tile sizes

In [3]:
if GENERATE_TILES:
    pan_size_dir_paths = []
    for i in range(len(MS_SIZES)):
        meta = allocate_tiles(meta, by_partition=False, n_tiles_total=PAN_N_TILES[i])
        ms_size = MS_SIZES[i]
        pan_size = PAN_SIZES[i]
        pan_size_dir_path = str(DATA_PATH_TILES+'/'+str(pan_size))
        pan_size_dir_paths.append(pan_size_dir_path)
        generate_all_tiles(meta, pan_size_dir_path, ms_height_width=[ms_size, ms_size], 
                           sr_factor=SR_FACTOR, save_by_partition=False,
                           cloud_sea_removal=False,
                           print_tile_info=False, save_meta_to_disk=True)

# Loading the MATLAB Engine for Python

In [4]:
if CALCULATE_NIQE_MA:
    matlab_engine = MatLabEngine('modules/matlab', ma=True, niqe=True, 
                                 output_dtype='uint16', shave_width=SHAVE_WIDTH)

Starting matlab.engine ...
matlab.engine started


# Pandas Dataframe to store results
`tile_path` is set as index.

In [5]:
results = pd.DataFrame()
paths_all_tiles = [path.as_posix() for path in list_tiles_in_dir(DATA_PATH_TILES, ms_or_pan='pan')]
results['tile_path'] = paths_all_tiles
results.set_index('tile_path', inplace=True)
results

Found 18028 tiles of type pan in the directory provided


""
tile_path
data/niqe-ma-study/1024/GE01_La_Spezia_2009_09_25_011651186010_0/pan/00000.tif
data/niqe-ma-study/1024/GE01_La_Spezia_2009_09_25_011651186010_0/pan/00001.tif
data/niqe-ma-study/1024/GE01_La_Spezia_2009_09_25_011651186010_0/pan/00002.tif
data/niqe-ma-study/1024/GE01_La_Spezia_2009_09_25_011651186010_0/pan/00003.tif
data/niqe-ma-study/1024/GE01_La_Spezia_2009_09_25_011651186010_0/pan/00004.tif
...
data/niqe-ma-study/896/WV02_Toulon_2019_12_15_011650875010_0/pan/00009.tif
data/niqe-ma-study/896/WV02_Toulon_2019_12_15_011650875010_0/pan/00010.tif
data/niqe-ma-study/896/WV02_Toulon_2019_12_15_011650875010_0/pan/00011.tif


# Actual NIQE and Ma calculation

Only preprocessing step is shaving 4 pixels from the borders of every tile, since this also is done in the main experiments as well as in the 2018 PIRM SISR Challenge.

In [6]:
if CALCULATE_NIQE_MA:
    for i, pan_size_dir_path in enumerate(pan_size_dir_paths):
        tile_size = PAN_SIZES[i]
        pan_geotiff_tile_paths = list_tiles_in_dir(pan_size_dir_path, ms_or_pan='pan')

        pan_tiles = []
        for pan_geotiff_tile_path in pan_geotiff_tile_paths:
            pan_tile = geotiff_to_ndarray(pan_geotiff_tile_path)
            pan_tiles.append(pan_tile)

        for j in range(len(pan_tiles)):
            niqes_ml, niqes_sk, mas = None, None, None
            batch_range = slice(BATCH_SIZE*j, BATCH_SIZE*(j+1))
            pan_tiles_batch = np.array(pan_tiles[batch_range])
            if pan_tiles_batch.shape[0] == 0:
                break

            tile_paths_batch = pan_geotiff_tile_paths[batch_range]

            n_tiles = pan_tiles_batch.shape[0]
            print('Tile size', tile_size, ', shape:', pan_tiles_batch.shape, ', batch:', j, 
                  ', batch size:', n_tiles, ', shave width:', matlab_engine.shave_width)

            tic = time.perf_counter()
            niqes_ml = matlab_engine.matlab_niqe_metric(pan_tiles_batch)
            toc = time.perf_counter()
            niqe_ml_sec_per_tile = (toc-tic)/n_tiles
            print('NIQE (matlab) mean:', np.mean(niqes_ml), ', sd:',  np.std(niqes_ml), 
                  ', seconds per tile:', niqe_ml_sec_per_tile)

            if tile_size > 192:
                tic = time.perf_counter()
                niqes_sk = matlab_engine.skvideo_niqe_metric(pan_tiles_batch)
                toc = time.perf_counter()
                niqe_sk_sec_per_tile = (toc-tic)/n_tiles
                print('NIQE (skvideo) mean:', np.mean(niqes_sk), ', sd:',  np.std(niqes_sk), 
                      ', seconds per tile:', niqe_sk_sec_per_tile)

            tic = time.perf_counter()
            mas = matlab_engine.matlab_ma_metric(pan_tiles_batch)
            toc = time.perf_counter()
            ma_sec_per_tile = (toc-tic)/n_tiles
            print('Ma (matlab) mean:', np.mean(mas), ', sd:',  np.std(mas), 
                  ', seconds per tile:', ma_sec_per_tile)
            tile_paths_batch = [path.as_posix() for path in tile_paths_batch]
            for i, tile_path in enumerate(tile_paths_batch):
                try:
                    results.loc[tile_path, 'niqe_ml'] = niqes_ml[i]
                    results.loc[tile_path, 'niqe_ml_s_per_tile'] = niqe_ml_sec_per_tile
                except TypeError:
                    pass
                try:
                    results.loc[tile_path, 'niqe_sk'] = niqes_sk[i]
                    results.loc[tile_path, 'niqe_sk_s_per_tile'] = niqe_sk_sec_per_tile
                except TypeError:
                    pass
                try:
                    results.loc[tile_path, 'ma'] = mas[i]
                    results.loc[tile_path, 'ma_s_per_tile'] = ma_sec_per_tile
                except TypeError:
                    pass

        csv_path = pathlib.Path(DATA_PATH_TILES).joinpath('results-'+str(tile_size)+'.csv')
        results.to_csv(csv_path)
        print('Saved results dataframe as csv at', csv_path.as_posix())        

NameError: name 'pan_size_dir_paths' is not defined

# Inspect results

We read the last of the `.csv`s from the previous experiment and derive some more statistics from the existing dataframe columns.

In [ ]:
results = pd.read_csv(DATA_PATH_TILES+'/results-1024.csv')

# Derive tile size from the path string
def tile_size_from_path(row):
    p = row.iloc[0]
    p = pathlib.Path(row.iloc[0])
    tile_size = int(p.parents[2].stem)
    row['tile_size'] = tile_size
    row['shave_width'] = SHAVE_WIDTH
    row['tile_size_shaved'] = tile_size - 2*SHAVE_WIDTH
    return row

results = results.apply(tile_size_from_path, axis=1)

# Calculate Perceptual Index (PI)
results['pi'] = perceptual_index(ma=results['ma'], niqe=results['niqe_ml']).numpy()

# Calculate compute time per fixed area size
def compute_time_per_area(results, metrics=('niqe_ml', 'ma'), area_size=100*100):
    for metric in metrics:
        tile_size_area = results['tile_size'] ** 2
        results[metric+'_s_per_'+str(area_size)+'_pixels'] = area_size * (results[metric+'_s_per_tile'] / tile_size_area)
    return results
results = compute_time_per_area(results, area_size=100*100)
results

# Plot 1: How our metrics vary by tile size

Note: Perceptual Index (PI) is simply a function of Ma and NIQE:

$\textrm{PI} = \frac{1}{2}\left(\left(10-\textrm{Ma}\right) + \textrm{NIQE}\right)$

- NIQE: Lower is better. 0 is best.
- Ma et al.: Higher is better. 10 is best
- PI: Lower is better. 0 is best

In [ ]:
# Aggregated statistics. Grouped by tile_size:
results_medians = results.groupby('tile_size').median()
results_means = results.groupby('tile_size').mean()
results_stds = results.groupby('tile_size').std()
results_counts = results.groupby('tile_size').count().iloc[:,0]

fig = plotly.subplots.make_subplots(rows=3,
                                    subplot_titles=('NIQE (MATLAB)', 
                                                    'Ma et al. (MATLAB)', 
                                                    'Perceptual Index (PI)'),
                                    shared_xaxes=True,
                                    vertical_spacing=0.05)
for i, score in enumerate(['niqe_ml', 'ma', 'pi']):
    if i == 0:
        showlegend = True
    else:
        showlegend = False
    fig.add_box(x=results['tile_size'], y=results[score], 
                name='scores',
                marker=dict(color=px.colors.qualitative.Plotly[0]),
                legendgroup='scores',
                showlegend=showlegend,
                text=results_counts.values,
                row=i+1, col=1)
    fig.add_scatter(x=results_medians.index.values, y=results_medians[score].values,
                    name='median',
                    line=dict(color=px.colors.qualitative.Plotly[1]),
                    legendgroup='median',
                    showlegend=showlegend, 
                    row=i+1, col=1)
    fig.add_scatter(x=results_medians.index.values, y=results_means[score].values,
                    name='mean',
                    line=dict(color=px.colors.qualitative.Plotly[2]),
                    legendgroup='mean',
                    showlegend=showlegend,
                    row=i+1, col=1)
    
fig.update_layout(autosize=True,
                  title= 'NIQE and Ma et al. Satellite Tile Size Study - Scores',
                  height=1000)
fig.update_yaxes(range=[2,10], row=1, col=1)
fig.update_yaxes(range=[2,6], row=2, col=1)
fig.update_yaxes(range=[2,10], row=3, col=1)
fig.update_xaxes(showgrid=True, tickvals=results_counts.index.values)

fig.show()

NIQE is unstable at small tile sizes, basically non-functional at tile sizes 128 and 192. This directly translates to the PI plot since PI is a function of NIQE. Ma et al. on the other hand is more stable, slowly converging to a mean slightly around 4.1.

# Plot 2: How compute time of metrics vary by tile size

We do two comparisons here. In the first row we compare compute time per tile for NIQE and Ma. In the second row we have *normalized* compute time so that the unit is seconds per `100x100` tile. By doing this we can see if the dimensions of the tile itself affects compute time.

In [ ]:
fig = plotly.subplots.make_subplots(rows=2, cols=2,
                                    subplot_titles=('NIQE - seconds per tile', 
                                                    'Ma et al. - seconds per tile', 
                                                    'NIQE - seconds per 100x100 tile', 
                                                    'Ma et al. - seconds per 100x100 tile'),
                                    shared_xaxes=True,
                                    vertical_spacing=0.075, horizontal_spacing=0.075)
showlegend = True
for j, metric in enumerate(['niqe_ml', 'ma']):
    for i, per_what in enumerate(['_s_per_tile', '_s_per_10000_pixels']):
        if i == 1:
            showlegend = False
        seconds_per = metric + per_what
        fig.add_box(x=results['tile_size'], y=results[seconds_per], 
                    name='compute time',
                    marker=dict(color=px.colors.qualitative.Plotly[0]),
                    legendgroup='compute',
                    showlegend=showlegend,
                    text=results_counts.values,
                    row=i+1, col=j+1)
        fig.add_scatter(x=results_medians.index.values, y=results_means[seconds_per].values,
                        name='mean compute time',
                        line=dict(color=px.colors.qualitative.Plotly[2]),
                        legendgroup='mean_compute',
                        showlegend=showlegend,
                        row=i+1, col=j+1)
    
fig.update_layout(autosize=True,
                  title= 'NIQE and Ma et al. Satellite Tile Size Study - Compute time per tile ',

                  height=1000)
#fig.update_yaxes(range=[2,10], row=1, col=1)
fig.update_yaxes(range=[0,0.01], row=2, col=1)
fig.update_yaxes(range=[0.4,0.7], row=2, col=2)
fig.update_xaxes(showgrid=True, tickvals=results_counts.index.values)

fig.show()

The top two plots unsurprisingly shows an increase of compute time as the size of the tiles increase. An initial assessment before having done a proper time complexity study is that both runs in polynomial time. Calculating the Ma score is definitely more time consuming, around two orders of magnitude.

When comparing the two bottom plots, those where compute time is normalized to a 100x100 square tile, we first see that computing scores for very small tile sizes are inefficient. This can be because of overhead costs. From approximately 256 and upwards we see what may look like a linear slight trend upward.


TODO for the thesis:
- Time complexity analysis to estimate Big-O as a function of tile size

# Visual inspection

Setting tile size to 512 we inspect the image tiles scoring worst and best for all three metrics. This gives us an impression of how the metrics work on our dataset.

In [ ]:
K_BEST_WORST = 25
def k_best_worst_per_tile_size(results, best_worst='best', tile_size=512, metric='ma', k=8):
    top_results = results.loc[results['tile_size'] == tile_size, :]
    if metric == 'ma' and best_worst == 'best':
        ascending = False
    elif metric == 'ma' and best_worst == 'worst':
        ascending = True
    elif metric in ['niqe_ml', 'pi'] and best_worst == 'best':
        ascending = True
    elif metric in ['niqe_ml', 'pi'] and best_worst == 'worst':
        ascending = False
    else:
        raise NotImplementedError
        
    top_results = top_results.sort_values(metric, ascending=ascending)
    top_results = top_results.iloc[:k, :]
    return top_results

best_ma_results = k_best_worst_per_tile_size(results, best_worst='best', tile_size=512, metric='ma', k=K_BEST_WORST)
#best_ma_results
worst_ma_results = k_best_worst_per_tile_size(results, best_worst='worst', tile_size=512, metric='ma', k=K_BEST_WORST)
#worst_ma_results
best_niqe_results = k_best_worst_per_tile_size(results, best_worst='best', tile_size=512, metric='niqe_ml', k=K_BEST_WORST)
#best_niqe_results
worst_niqe_results = k_best_worst_per_tile_size(results, best_worst='worst', tile_size=512, metric='niqe_ml', k=K_BEST_WORST)
#worst_niqe_results
best_pi_results = k_best_worst_per_tile_size(results, best_worst='best', tile_size=512, metric='pi', k=K_BEST_WORST)
#best_pi_results
worst_pi_results = k_best_worst_per_tile_size(results, best_worst='worst', tile_size=512, metric='pi', k=K_BEST_WORST)
#worst_pi_results

In [ ]:
def plot_images(imgs, cols=None, fig_size=1000, spacing=0.0, zmin=0, zmax=255, stretch=True):
    if tf.is_tensor(imgs):
        imgs = imgs.numpy()
    n = imgs.shape[0]
    if cols is None:
        cols = int(np.ceil(np.sqrt(n)))
    rows = int(np.ceil(n / cols))
    fig = plotly.subplots.make_subplots(rows=rows, cols=cols,
                                        horizontal_spacing=spacing,
                                        vertical_spacing=spacing, 
                                        shared_xaxes=False, shared_yaxes=False)
    k = 0
    for i in range(rows):
        for j in range(cols):
            if k == n:
                break
            #print(i,j,k)
            img = imgs[k,:,:,0]
            if stretch:
                # stretch through +- 5 std from mean
                mean = np.mean(img)
                std = np.std(img)
                zmin = max(0, mean - 5 * std)
                zmax = mean + 5 * std
            fig.add_trace(go.Heatmap(z=img, 
                                     colorscale='gray', 
                                     zmin=zmin, 
                                     zmax=zmax,
                                     showscale=False),
                          row=i+1, col=j+1)
            k += 1

    size_scale = fig_size / (cols * 250)
    fig.update_layout(height=size_scale*rows*250, 
                      width=fig_size)
    fig.update_yaxes(visible=False)
    fig.update_xaxes(visible=False)
    fig.show()

## Best Ma 

We see that Ma rewards some sea tiles where it looks like the sea has a high fidelity surface (lots of structure).

In [ ]:
imgs = geotiff_to_ndarray(best_ma_results['tile_path'].to_list())
plot_images(imgs, fig_size=1000, spacing=0.00, zmin=0, zmax=get_max_uint_from_bit_depth(11), stretch=True)

# Best NIQE

It looks like NIQE prefers clear and varied land imagery tiles.

In [ ]:
imgs = geotiff_to_ndarray(best_niqe_results['tile_path'].to_list())
plot_images(imgs, fig_size=1000, spacing=0.00, zmin=0, zmax=get_max_uint_from_bit_depth(11), stretch=True)

## Best Perceptual Index (PI)

On first glance it is a little hard to differentiate these results from NIQE.

In [ ]:
imgs = geotiff_to_ndarray(best_pi_results['tile_path'].to_list())
plot_images(imgs, fig_size=1000, spacing=0.00, zmin=0, zmax=get_max_uint_from_bit_depth(11), stretch=True)

## Worst Ma
Clouds and sea without interesting structure.

In [ ]:
imgs = geotiff_to_ndarray(worst_ma_results['tile_path'].to_list())
plot_images(imgs, fig_size=1000, spacing=0.00, zmin=0, zmax=get_max_uint_from_bit_depth(11), stretch=True)

## Worst NIQE

Similar results as for Ma

In [ ]:
imgs = geotiff_to_ndarray(worst_niqe_results['tile_path'].to_list())
plot_images(imgs, fig_size=1000, spacing=0.00, zmin=0, zmax=get_max_uint_from_bit_depth(11), stretch=True)

## Worst Perceptual Index (PI)

Similar to Ma and NIQE.

In [ ]:
imgs = geotiff_to_ndarray(worst_pi_results['tile_path'].to_list())
plot_images(imgs, fig_size=1000, spacing=0.00, zmin=0, zmax=get_max_uint_from_bit_depth(11), stretch=True)